In [1]:
from translate import *

df = pd.read_csv("02.csv")
df['translate_claude'] = df.apply(lambda x: "NA", axis=1)
df['revision_claude'] = df.apply(lambda x: "NA", axis=1)
for i in tqdm(range(len(df))):
    text = df['Transcript'].iloc[i]
    translated_text, revision = get_translate(text)
    df.at[i, 'translate_claude'] = translated_text  
    df.at[i, 'revision_claude'] = revision 
df.to_csv("02_translate_claude_v2.csv", index=False)

 12%|█▎        | 1/8 [00:13<01:35, 13.65s/it]

In [ ]:
import glob
file_names = glob.glob("data/*.csv")
for file_name in file_names:
    df = pd.read_csv(file_name)
    df['translate_claude'] = df.apply(lambda x: "NA", axis=1)
    df['revision_claude'] = df.apply(lambda x: "NA", axis=1)
    for i in tqdm(range(len(df))):
        text = df['Transcript'].iloc[i]
        translated_text, revision = get_translate(text)
        df.at[i, 'translate_claude'] = translated_text  
        df.at[i, 'revision_claude'] = translated_text  
    df.to_csv(file_name.replace("data", "data_translate_claude"), index=False)